In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from adtk.visualization import plot
from adtk.detector import ThresholdAD
from pandas.plotting import scatter_matrix

In [18]:
dataframe = pd.read_csv("cardox.csv", index_col="date")
dataframe

,value
date,
1958-03-01,315.71
1958-04-01,317.45
1958-05-01,317.50
1958-06-01,317.10
1958-07-01,315.86
...,...
2018-07-01,408.71
2018-08-01,406.99
2018-09-01,405.51


In [19]:
threshold_ad = ThresholdAD(high=2800, low=800)
anomalies = threshold_ad.detect(dataframe)
plot(dataframe, anomaly=anomalies, ts_linewidth=1, ts_markersize=3, anomaly_markersize=5, anomaly_color='red', anomaly_tag="marker");

TypeError: Index of the input time series must be a pandas DatetimeIndex object.